
# Banking System Simulation

This notebook integrates the entire banking system code and demonstrates its functionalities, including account creation, 
depositing and withdrawing funds, checking balances, and viewing transaction history.


In [3]:

import os
import csv
import random

def load_user_data(file_name="transactions.csv"):
    """Load user transaction data from a file."""
    users_data = {}
    try:
        if os.path.exists(file_name):
            with open(file_name, "r") as file:
                reader = csv.DictReader(file)
                for row in reader:
                    name = row["name"]
                    users_data[name] = {
                        "pin": int(row["pin"]),
                        "balance": float(row["balance"]),
                        "transactions": eval(row["transactions"]),
                    }
    except (OSError, IOError, KeyError, ValueError):
        print("Oops, something went wrong while loading the transaction file. Using default data.")
    return users_data

def save_user_data(users_data, file_name="transactions.csv"):
    """Save the current user data to a file."""
    try:
        with open(file_name, "w", newline="") as file:
            fieldnames = ["name", "pin", "balance", "transactions"]
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for name, details in users_data.items():
                writer.writerow({
                    "name": name,
                    "pin": details["pin"],
                    "balance": details["balance"],
                    "transactions": details["transactions"],
                })
    except (OSError, IOError):
        print("There was an error saving your data. Please try again later.")

# Load all user accounts at the start
users_data = load_user_data()

def generate_new_pin():
    """Generate a new unique PIN that isn't already taken."""
    existing_pins = {details["pin"] for details in users_data.values()}
    while True:
        pin = random.randint(1000, 9999)
        if pin not in existing_pins:
            return pin

def create_user_account(name):
    """Create a new user account with a unique PIN."""
    if name in users_data:
        return f"An account already exists for {name}. Please log in instead."
    pin = generate_new_pin()
    users_data[name] = {"pin": pin, "balance": 0.0, "transactions": []}
    save_user_data(users_data)
    return f"Account created for {name} with PIN {pin}. Keep it secure!"

def validate_pin(name, pin):
    """Verify if the entered PIN matches the one associated with the account."""
    return users_data[name]["pin"] == pin

def deposit_funds(name, amount):
    """Deposit funds into a user's account."""
    if name not in users_data:
        return "Account not found. Please check the name and try again."
    if amount <= 0:
        return "Deposit amount must be a positive number."
    users_data[name]["balance"] += amount
    users_data[name]["transactions"].append({"type": "Deposit", "amount": amount, "balance": users_data[name]["balance"]})
    save_user_data(users_data)
    return f"${amount} has been deposited into your account. Your new balance is: ${users_data[name]['balance']}."

def withdraw_funds(name, amount):
    """Withdraw funds from a user's account."""
    if name not in users_data:
        return "Account not found. Please check the name and try again."
    if amount <= 0:
        return "Withdrawal amount must be positive."
    if users_data[name]["balance"] < amount:
        return "Insufficient balance for this withdrawal."
    users_data[name]["balance"] -= amount
    users_data[name]["transactions"].append({"type": "Withdrawal", "amount": amount, "balance": users_data[name]["balance"]})
    save_user_data(users_data)
    return f"${amount} has been withdrawn from your account. Your new balance is: ${users_data[name]['balance']}."

def check_account_balance(name):
    """Check the balance of a user's account."""
    if name not in users_data:
        return "Account not found. Please check the name and try again."
    return f"Your current balance is: ${users_data[name]['balance']}."

def view_account_statement(name):
    """View the transaction history for a user's account."""
    if name not in users_data:
        return "Account not found. Please check the name and try again."
    transactions = users_data[name]["transactions"]
    if not transactions:
        return f"No transactions available for {name}."
    statement = [f"Account statement for {name}:"]
    for transaction in transactions:
        statement.append(f"- {transaction['type']}: ${transaction['amount']}. Balance: ${transaction['balance']}")
    return "\n".join(statement)


## Create New Account

In [5]:

# Create a new user account
new_account_name = "Alice"
print(create_user_account(new_account_name))


Account created for Alice with PIN 2290. Keep it secure!


## Deposit Funds

In [7]:

# Deposit funds into the account
print(deposit_funds(new_account_name, 1000))


$1000 has been deposited into your account. Your new balance is: $1000.0.


## Withdraw Funds

In [9]:

# Withdraw funds from the account
print(withdraw_funds(new_account_name, 200))


$200 has been withdrawn from your account. Your new balance is: $800.0.


## Check Balance

In [11]:

# Check the account balance
print(check_account_balance(new_account_name))


Your current balance is: $800.0.


## View Account Statement

In [13]:

# View the account statement
print(view_account_statement(new_account_name))


Account statement for Alice:
- Deposit: $1000. Balance: $1000.0
- Withdrawal: $200. Balance: $800.0
